In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from time import sleep
import re

In [2]:
def get_basic_info():
    base_url="https://baike.baidu.com"
    url ="https://baike.baidu.com/item/%E4%B8%AD%E5%9B%BD%E4%BA%BA%E6%B0%91%E8%A7%A3%E6%94%BE%E5%86%9B%E5%B0%86%E5%B8%85%E5%90%8D%E5%BD%95/1065859?fr=aladdin"
    html=urlopen(url).read().decode("utf-8")
    soup=BeautifulSoup(html,features="lxml")
    return soup
soup=get_basic_info()
#soup.find_all(class_="basicInfo-item value")

In [3]:
def get_list():
    all_list=[]
    for i in range(3,9):
        for j in soup.find_all(class_="para")[i].contents:
            if j.string!=' ':
                all_list.append(j.string.strip(" "))
    #all_list
    out_index_list=[]
    for i in range(3,9):
        for j in soup.find_all(class_="para")[i].contents:
            if "%" not in str(j) and j!=" ":
                out_index_list.append(j.string.strip(" "))
    #out_index_list
    return all_list,out_index_list
all_list,out_index_list=get_list()
print("总列表是%i,在外面的列表是%i"%(len(all_list),len(out_index_list)))

总列表是1002,在外面的列表是175


In [4]:
#异常数据处理
print(out_index_list.index('（56年1月25日授予）李聚奎 （58年4月授予）'))
print(out_index_list.index('炳 黄 远 吴 涛'))
print(out_index_list.index('叁 陈 钦 谢 斌'))
print(out_index_list.index('一 邓兆祥 罗 斌 苏 进'))
print(out_index_list.index('王 屏 姚 国民'))
out_index_list[16]="李聚奎"
out_index_list[48]="黄 远 吴 涛"
out_index_list[71]="陈 钦 谢 斌"
out_index_list[87]="邓兆祥 罗 斌 苏 进"
out_index_list[163]="王 屏 姚国民"

16
48
71
87
163


In [5]:
def get_clean_list():
    clean_index_list=[]
    for i in range(len(out_index_list)):
        if "·" in out_index_list[i]:
            continue
        for j in range(len(out_index_list[i])//3):
            clean_index_list.append(out_index_list[i][0+j*3:4+j*3])
    clean_index_list.append("马尔果甫·伊斯卡果夫")
    clean_index_list.append("孙 光")
    clean_index_list.append('祖农·太也甫夫')
    return clean_index_list
clean_index_list=get_clean_list()

In [6]:
len(clean_index_list)

215

In [7]:
from selenium import webdriver
from time import sleep
#driver = webdriver.Chrome(r'C:\Users\HASEE\chromedriver.exe')
driver = webdriver.Chrome(r'C:\Users\HASEE\chromedriver.exe')
    driver.get("https://baike.baidu.com/")
clean_list=[]

for i in range(len(clean_index_list)):
    driver.find_element_by_id('query').send_keys(clean_index_list[i])
        # 点击搜索按钮
    driver.find_element_by_id('search').click()
    clean_list.append(driver.current_url)
    sleep(0.01)
    driver.back()
    '''
    try:
        if driver.find_element_by_class_name("item"):
            find=driver.find_element_by_xpath("//a[contains(text(),'将')]")
        #获取当前的URL的地址
            clean_list.append(driver.current_url)
            #driver.back()

        sleep(3)
    except:
        clean_list.append(driver.current_url)
        print(clean_list)
        #pass
    '''
    #print(clean_index_list[i])
    #driver.back()
    #continue

#关闭浏览器
#driver.close()
    

In [12]:
sub_urls = soup.find_all("a", {"target": "_blank", "href": re.compile("/item/(%.{2})")})
sub_urls=sub_urls[:len(sub_urls)-2]
click_url=["https://baike.baidu.com"+i['href'] for i in sub_urls]
click_url.extend(clean_list)
print("已有的链接地址%i"%len(click_url))

已有的链接地址1042


In [13]:
import pandas as pd
content=["北伐战争","土地革命","解放战争","抗日战争","解放后"]
all_info=[]
for i in range(len(click_url)):
    url=click_url[i]
    html=urlopen(url).read().decode("utf-8")
    soup=BeautifulSoup(html,features="lxml")
    basic_info=[]
    for j in soup.find_all(class_="basicInfo-item"):
        basic_info.append(str(j.string).replace("\xa0","").replace("\n",""))
        basic_info_name=basic_info[::2]
        basic_info_value=basic_info[1::2]
        info_dict=dict(zip(basic_info_name,basic_info_value))
    text={}        
    for j in soup.find_all("div",{"class":"para","label-module":"para"}):
            for k in content:
                if k in j.get_text():
                    if text.get(k,None)==None:
                        text[k]=j.get_text().replace("\n","").replace("\xa0","")
                        #continue
                    else:
                        text[k]+=j.get_text().replace("\n","").replace("\xa0","")
                        #continue
                else:
                    pass
    info_dict.update(text)
    all_info.append(info_dict)
    print("采集完第%i个将军的信息!"%i)


采集完第0个将军的信息!
采集完第1个将军的信息!
采集完第2个将军的信息!
采集完第3个将军的信息!
采集完第4个将军的信息!
采集完第5个将军的信息!
采集完第6个将军的信息!
采集完第7个将军的信息!
采集完第8个将军的信息!
采集完第9个将军的信息!
采集完第10个将军的信息!
采集完第11个将军的信息!
采集完第12个将军的信息!
采集完第13个将军的信息!
采集完第14个将军的信息!
采集完第15个将军的信息!
采集完第16个将军的信息!
采集完第17个将军的信息!
采集完第18个将军的信息!
采集完第19个将军的信息!
采集完第20个将军的信息!
采集完第21个将军的信息!
采集完第22个将军的信息!
采集完第23个将军的信息!
采集完第24个将军的信息!
采集完第25个将军的信息!
采集完第26个将军的信息!
采集完第27个将军的信息!
采集完第28个将军的信息!
采集完第29个将军的信息!
采集完第30个将军的信息!
采集完第31个将军的信息!
采集完第32个将军的信息!
采集完第33个将军的信息!
采集完第34个将军的信息!
采集完第35个将军的信息!
采集完第36个将军的信息!
采集完第37个将军的信息!
采集完第38个将军的信息!
采集完第39个将军的信息!
采集完第40个将军的信息!
采集完第41个将军的信息!
采集完第42个将军的信息!
采集完第43个将军的信息!
采集完第44个将军的信息!
采集完第45个将军的信息!
采集完第46个将军的信息!
采集完第47个将军的信息!
采集完第48个将军的信息!
采集完第49个将军的信息!
采集完第50个将军的信息!
采集完第51个将军的信息!
采集完第52个将军的信息!
采集完第53个将军的信息!
采集完第54个将军的信息!
采集完第55个将军的信息!
采集完第56个将军的信息!
采集完第57个将军的信息!
采集完第58个将军的信息!
采集完第59个将军的信息!
采集完第60个将军的信息!
采集完第61个将军的信息!
采集完第62个将军的信息!
采集完第63个将军的信息!
采集完第64个将军的信息!
采集完第65个将军的信息!
采集完第66个将军的信息!
采集完第67个将军的信息!
采集完第68个将军的信息!
采集完第69个将军的信息!
采集完第70个将军的信息!
采集完第71个将军的信息!
采集

采集完第554个将军的信息!
采集完第555个将军的信息!
采集完第556个将军的信息!
采集完第557个将军的信息!
采集完第558个将军的信息!
采集完第559个将军的信息!
采集完第560个将军的信息!
采集完第561个将军的信息!
采集完第562个将军的信息!
采集完第563个将军的信息!
采集完第564个将军的信息!
采集完第565个将军的信息!
采集完第566个将军的信息!
采集完第567个将军的信息!
采集完第568个将军的信息!
采集完第569个将军的信息!
采集完第570个将军的信息!
采集完第571个将军的信息!
采集完第572个将军的信息!
采集完第573个将军的信息!
采集完第574个将军的信息!
采集完第575个将军的信息!
采集完第576个将军的信息!
采集完第577个将军的信息!
采集完第578个将军的信息!
采集完第579个将军的信息!
采集完第580个将军的信息!
采集完第581个将军的信息!
采集完第582个将军的信息!
采集完第583个将军的信息!
采集完第584个将军的信息!
采集完第585个将军的信息!
采集完第586个将军的信息!
采集完第587个将军的信息!
采集完第588个将军的信息!
采集完第589个将军的信息!
采集完第590个将军的信息!
采集完第591个将军的信息!
采集完第592个将军的信息!
采集完第593个将军的信息!
采集完第594个将军的信息!
采集完第595个将军的信息!
采集完第596个将军的信息!
采集完第597个将军的信息!
采集完第598个将军的信息!
采集完第599个将军的信息!
采集完第600个将军的信息!
采集完第601个将军的信息!
采集完第602个将军的信息!
采集完第603个将军的信息!
采集完第604个将军的信息!
采集完第605个将军的信息!
采集完第606个将军的信息!
采集完第607个将军的信息!
采集完第608个将军的信息!
采集完第609个将军的信息!
采集完第610个将军的信息!
采集完第611个将军的信息!
采集完第612个将军的信息!
采集完第613个将军的信息!
采集完第614个将军的信息!
采集完第615个将军的信息!
采集完第616个将军的信息!
采集完第617个将军的信息!
采集完第618个将军的信息!
采集完第619个将军的信息!
采集完第620个将军

采集完第1094个将军的信息!
采集完第1095个将军的信息!
采集完第1096个将军的信息!
采集完第1097个将军的信息!
采集完第1098个将军的信息!
采集完第1099个将军的信息!
采集完第1100个将军的信息!
采集完第1101个将军的信息!
采集完第1102个将军的信息!
采集完第1103个将军的信息!
采集完第1104个将军的信息!
采集完第1105个将军的信息!
采集完第1106个将军的信息!
采集完第1107个将军的信息!
采集完第1108个将军的信息!
采集完第1109个将军的信息!
采集完第1110个将军的信息!
采集完第1111个将军的信息!
采集完第1112个将军的信息!
采集完第1113个将军的信息!
采集完第1114个将军的信息!
采集完第1115个将军的信息!
采集完第1116个将军的信息!
采集完第1117个将军的信息!
采集完第1118个将军的信息!
采集完第1119个将军的信息!
采集完第1120个将军的信息!
采集完第1121个将军的信息!
采集完第1122个将军的信息!
采集完第1123个将军的信息!
采集完第1124个将军的信息!
采集完第1125个将军的信息!
采集完第1126个将军的信息!
采集完第1127个将军的信息!
采集完第1128个将军的信息!
采集完第1129个将军的信息!
采集完第1130个将军的信息!
采集完第1131个将军的信息!
采集完第1132个将军的信息!
采集完第1133个将军的信息!
采集完第1134个将军的信息!
采集完第1135个将军的信息!
采集完第1136个将军的信息!
采集完第1137个将军的信息!
采集完第1138个将军的信息!
采集完第1139个将军的信息!
采集完第1140个将军的信息!
采集完第1141个将军的信息!
采集完第1142个将军的信息!
采集完第1143个将军的信息!
采集完第1144个将军的信息!
采集完第1145个将军的信息!
采集完第1146个将军的信息!


KeyboardInterrupt: 

In [4]:
url =click_url[10]
html=urlopen(url).read().decode("utf-8")
soup=BeautifulSoup(html,features="lxml")
basic_info=[]
for i in soup.find_all(class_="basicInfo-item"):
    basic_info.append(str(i.string).replace("\xa0","").replace("\n",""))
basic_info_name=basic_info[::2]
basic_info_value=basic_info[1::2]
info_dict=dict(zip(basic_info_name,basic_info_value))

In [16]:
info_dict

{'中文名': '黄克诚',
 '别名': '黄时瑄',
 '国籍': '中华人民共和国',
 '民族': '汉族',
 '出生地': 'None',
 '出生日期': '1902年10月1日',
 '逝世日期': '1986年12月28日',
 '职业': '军人',
 '毕业院校': '湖南省立第三师范学校',
 '信仰': '共产主义',
 '主要成就': 'None',
 '代表作品': 'None',
 '军衔': '大将（1955年）',
 '政治面貌': 'None',
 '北伐战争': '黄克诚（1902年10月1日-1986年12月28日），1902年10月1日出身于湖南省永兴县一个贫苦农民家庭。1925年加入中国共产党。1926年参加北伐战争。1928年在湘南起义中参与领导永兴年关暴动，并率部随朱德、陈毅上井冈山。曾任中国工农红军团长，师政委，军政治部主任，红三军团政治部代主任等职。黄克诚，原名黄时瑄[1]，1902年10月1日（清光绪廿八年壬寅岁八月三十）生于湖南省永兴县三塘乡下青村一个农民家庭。9岁起靠亲属们接济读了8年私塾。1920年入县立高等小学。1922年考入衡阳湖南省立第三师范学校，开始接触马克思主义，积极参加爱国学生运动。1925年加入中国共产党。1926年初入国共两党合办的广州中央政治讲习班，结业后参加北伐战争，在国民革命军第8军任营、团政治指导员，随部转战长沙、武汉及河南、江西、安徽等地。[2]1927年大革命失败后，黄克诚奉命回家乡秘密从事农运工作。1928年初北伐战争时期的黄克诚[3]参加湘南起义，组织农民暴动并攻占永兴县城，任永兴红色警卫团党代表兼参谋长，对湘南特委的“左”倾盲动政策进行过坚决抵制，并指责为右倾机会主义而受到排挤。后随朱德、陈毅上井冈山，任中国工农革命军第4军第12师第35团团长。不久改任湘南农军第2路游击司令，率部返回湘南活动。但途中队伍失散，他在家乡隐蔽了一段时间后，辗转到上海寻找党组织。',
 '抗日战争': '抗日战争初期黄克诚（左一）同邓小平(5张)抗日战争爆发时，黄克诚任八路军总政治部组织部部长。在深入部队调查研究后，向中央建议恢复被取消的政治委员制度，以保持人民军队的光荣传统和优良作风，很快被采纳。不久改任八路军第115师344旅政治委员，同旅长徐海东率部开展敌后游击战争，参加晋察冀边区反“八

In [15]:
info_dict.update(text)

In [13]:
text={}
content=["北伐战争","土地革命","解放战争","抗日战争",]
for i in soup.find_all("div",{"class":"para","label-module":"para"}):
    for j in content:
        if j in i.get_text():
            if text.get(j,None)==None:
                text[j]=i.get_text().replace("\n","").replace("\xa0","")
            else:
                text[j]+=i.get_text().replace("\n","").replace("\xa0","")
            #break
    else:
        pass

In [ ]:
soup.find_all("div",{"class":"para","label-module":"para"})

In [14]:
text

{'北伐战争': '黄克诚（1902年10月1日-1986年12月28日），1902年10月1日出身于湖南省永兴县一个贫苦农民家庭。1925年加入中国共产党。1926年参加北伐战争。1928年在湘南起义中参与领导永兴年关暴动，并率部随朱德、陈毅上井冈山。曾任中国工农红军团长，师政委，军政治部主任，红三军团政治部代主任等职。黄克诚，原名黄时瑄[1]，1902年10月1日（清光绪廿八年壬寅岁八月三十）生于湖南省永兴县三塘乡下青村一个农民家庭。9岁起靠亲属们接济读了8年私塾。1920年入县立高等小学。1922年考入衡阳湖南省立第三师范学校，开始接触马克思主义，积极参加爱国学生运动。1925年加入中国共产党。1926年初入国共两党合办的广州中央政治讲习班，结业后参加北伐战争，在国民革命军第8军任营、团政治指导员，随部转战长沙、武汉及河南、江西、安徽等地。[2]1927年大革命失败后，黄克诚奉命回家乡秘密从事农运工作。1928年初北伐战争时期的黄克诚[3]参加湘南起义，组织农民暴动并攻占永兴县城，任永兴红色警卫团党代表兼参谋长，对湘南特委的“左”倾盲动政策进行过坚决抵制，并指责为右倾机会主义而受到排挤。后随朱德、陈毅上井冈山，任中国工农革命军第4军第12师第35团团长。不久改任湘南农军第2路游击司令，率部返回湘南活动。但途中队伍失散，他在家乡隐蔽了一段时间后，辗转到上海寻找党组织。',
 '抗日战争': '抗日战争初期黄克诚（左一）同邓小平(5张)抗日战争爆发时，黄克诚任八路军总政治部组织部部长。在深入部队调查研究后，向中央建议恢复被取消的政治委员制度，以保持人民军队的光荣传统和优良作风，很快被采纳。不久改任八路军第115师344旅政治委员，同旅长徐海东率部开展敌后游击战争，参加晋察冀边区反“八路围攻”、晋东南反“九路围攻”，参与指挥温塘、张店、町店等战斗，为创建晋冀豫抗日根据地作出重要贡献。1940年2月起任八路军第2纵队政治委员、冀鲁豫军区司令员兼政治委员。5月率344旅和新编第2旅南下华中豫皖苏边区，同彭雪枫率领的新四军第6支队会合，组建八路军第4纵队并任政治委员。8月又奉命转赴皖东北，将淮河以北、津浦路以东由中共领导的数支部队统一整编为八路军第5纵队，任司令员兼政治委员。随即率部东进淮海，南下盐城、阜宁，同发起黄桥战役的新四军苏北部队在白驹镇会师，共同创建苏北抗日根据

In [ ]:
driver = webdriver.Chrome(r'C:\Users\HASEE\chromedriver.exe')
driver.get("https://baike.baidu.com/")
driver.find_element_by_id('query').send_keys(clean_index_list[-2])
        # 点击搜索按钮
driver.find_element_by_id('search').click()

In [ ]:
driver.find_element_by_xpath("//a[contains(text(),'将')]").click()

In [8]:
a={"a":1,"b":"a"}
a["a"]+=2

In [27]:
all_info

[{'中文名': '朱德',
  '国籍': '中华人民共和国',
  '民族': '汉',
  '出生地': '四川省仪陇县',
  '出生日期': '1886年12月1日',
  '逝世日期': '1976年7月6日',
  '信仰': '马列主义',
  '获得荣誉': '被授予中华人民共和国元帅军衔',
  '抗日战争': '\n\n\n\n\n在中共七大上讲演\n\n1937年7月抗日战争爆发后，任国民革命军第八路军总指挥（不久改称国民革命军第十八集团军总司令），率领八路军开赴华北前线，协同国民党军队对日作战，取得平型关等战斗的胜利，尔后指挥八路军各部深入敌后，开展游击战争，建立和扩大了许多抗日根据地。曾先后兼任第二战区东路军总指挥和第二战区副司令长官，指挥所辖部分国民党军队与八路军共同作战，维护了国共两党合作抗日的局面。\n\n\n\n\n朱德在山西武乡县\n[23]\n\n\n抗日战争全面爆发后，朱德同志怀着“与日寇决一死战，复我河山，保我民族”的决心，率领抗日将士挥师东渡黄河，开赴华北抗日前线。朱德同志坚决执行党的抗日民族统一战线政策，团结抗战，力挽狂澜，为挺进敌后、开辟华北抗日根据地作出了重大贡献。1940年5月，朱德同志从太行前线回到延安，协助毛泽东同志领导各敌后根据地的抗日战争。为克服陕甘宁边区的严重经济困难，朱德同志响应毛泽东同志“自己动手、丰衣足食”的号召，亲自指导和推动大生产运动，倡导培育了“南泥湾精神”，成为人民军队的一个光荣传统。1945年4月，朱德同志在党的七大上作《论解放区战场》的军事报告，从理论和实践的结合上系统总结了党领导抗日战争的基本经验。 解放战争中，朱德同志参与制定了“向北发展，向南防御”等一系列重大战略决策，协助毛泽东同志指挥各解放区军民，先后粉碎国民党反动派的全面进攻和重点进攻。朱德同志不畏艰险，深入前线，有效解决了各兵种协同作战、后勤补给、军工生产等问题。朱德同志亲自指导解放石家庄战役，为组织夺取大中城市的攻坚战积累了宝贵经验。在战略决战和战略追歼阶段，朱德同志协助毛泽东同志，运筹帷幄，决胜千里，指挥人民解放军彻底推翻了国民党反动派在中国大陆的统治，领导我们党团结全国各族人民建立了中华人民共和国。朱德同志为中国人民解放事业立下了丰功伟绩。',
  '解放战争': '在解放战争中，任中国人民解放军总司令。1945年4

In [15]:
import pandas as pd
df=pd.DataFrame(all_info)
df

,中文名,国籍,民族,出生地,出生日期,逝世日期,信仰,获得荣誉,抗日战争,解放战争,...,住户,生肖,参加红军,出生地点,身份,其他名称,部下,主要学说,第一任夫人,第二任夫人
0,朱德,中华人民共和国,汉,四川省仪陇县,1886年12月1日,1976年7月6日,马列主义,被授予中华人民共和国元帅军衔,在中共七大上讲演1937年7月抗日战争爆发后，任国民革命军第八路军总指挥（不久改称国民革命军...,在解放战争中，任中国人民解放军总司令。1945年4月，朱德同志在党的七大上作《论解放区战场》...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,彭德怀,中华人民共和国,汉族,湖南省湘潭县石潭镇乌石寨,1898年10月24日,1974年11月29日,共产主义,NaN,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,林彪,None,None,None,1907年12月5日,1971年9月13日,NaN,NaN,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,刘伯承,中国,汉族,重庆市开州区,1892年12月4日,1986年10月7日,共产主义,NaN,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,贺龙,中国,NaN,湖南桑植,1896年3月22日,1969年6月9日,NaN,NaN,长征胜利结束后的贺龙抗日战争开始后，红军改编为国民革命军第八路军，贺龙任八路军第120师师长...,贺龙被任命为一二零师师长日军投降后，贺龙率领晋绥部队主力挥师北上，解放了晋中广大地区，并与聂...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,李勃,NaN,NaN,NaN,650,716,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1143,余明,中国,汉族,四川,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1144,陈熙,中国,汉,NaN,1918年,1991年12月14日,NaN,NaN,NaN,陈熙（1918~1990.12.14）男，江西省赣州市兴国县兴莲乡官田村人。出生于贫农家庭。...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1145,陈熙,中国,汉,NaN,1918年,1991年12月14日,NaN,NaN,NaN,陈熙（1918~1990.12.14）男，江西省赣州市兴国县兴莲乡官田村人。出生于贫农家庭。...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df[["中文名","国籍"]]

None           69
军事学院           12
南京军事学院         10
抗日军政大学         10
陕北红军大学          7
               ..
陕北红军大学，军事学院     1
西北军第二军官学校       1
北洋大学            1
莫斯科东方大学军事班      1
成都工业学院，中法大学     1
Name: 毕业院校, Length: 145, dtype: int64

In [19]:
df=df.drop_duplicates("中文名")

In [67]:
df[["中文名","出生日期","逝世日期","出生地","毕业院校","北伐战争","土地革命","抗日战争","解放战争","解放后"]].to_csv("解放将军信息_01.csv",encoding="utf-8")

In [17]:
df=df.fillna("无")

In [21]:
df[["中文名","出生日期","逝世日期","出生地","毕业院校","北伐战争","土地革命","抗日战争","解放战争","解放后"]]

,中文名,出生日期,逝世日期,出生地,毕业院校,北伐战争,土地革命,抗日战争,解放战争,解放后
0,朱德,1886年12月1日,1976年7月6日,四川省仪陇县,无,无,1月，和毛泽东率红四军主力离开井冈山，向赣南、闽西进军。以武装斗争发动土地革命，建立红色政权...,在中共七大上讲演1937年7月抗日战争爆发后，任国民革命军第八路军总指挥（不久改称国民革命军...,在解放战争中，任中国人民解放军总司令。1945年4月，朱德同志在党的七大上作《论解放区战场》...,无
1,彭德怀,1898年10月24日,1974年11月29日,湖南省湘潭县石潭镇乌石寨,湖南陆军讲武堂,1926年，彭德怀随部队编入国民革命军，参加北伐战争，结识共产党人段德昌，开始接受共产主义思...,无,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,无
2,林彪,1907年12月5日,1971年9月13日,None,None,在黄埔军校由共青团转入中国共产党，并任3连中共支部书记。1926年10月毕业后，由广州北上武...,战役时间：土地革命战争时期的1930年11月5日至1931年1月3日，历时60天。战役时间：...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...
3,刘伯承,1892年12月4日,1986年10月7日,重庆市开州区,None,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,刘伯承与毛泽东主席天安门城楼上1949年4月参与指挥渡江战役，实施京沪杭会战，并亲自指挥第二...
4,贺龙,1896年3月22日,1969年6月9日,湖南桑植,无,1924年至1927年中国第一次国内革命战争期间，贺龙积极拥护孙中山先生“联俄、联共、扶助农...,土地革命战争时期，1927年参加领导南昌起义，任起义军总指挥，同年加入中国共产党。,长征胜利结束后的贺龙抗日战争开始后，红军改编为国民革命军第八路军，贺龙任八路军第120师师长...,贺龙被任命为一二零师师长日军投降后，贺龙率领晋绥部队主力挥师北上，解放了晋中广大地区，并与聂...,1949年12月，贺龙率华北野战军第十八兵团等部，由陕入川，配合刘伯承、邓小平指挥的第二野战...
...,...,...,...,...,...,...,...,...,...,...
1036,赖光勋,1914年11月,2003年1月25日17时,None,无,无,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,红军地图奇缺，有的地区甚至没有一份地图，赖光勋带领测绘科在极端艰苦的条件下，克服重重困难，千...
1037,黎光,1914年,无,四川省南充市仪陇县,None,无,无,抗日战争开始后，黎光任八路军129师386旅771团1营1连排长。在和日军作战中，一颗子弹从...,无,无
1038,汪易,1916年,1995年2月13日,四川省阆中市,无,无,无,无,汪易同志在解放战争时期，曾在承德、赤峰、热东、热辽的广大地区战斗、工作了3年时间，他在热河和...,无
1040,孙光,1961年6月21日,无,西安,无,无,无,无,无,无


In [23]:
df["解放后"].value_counts()

无                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                827
解放后，黎化南先后担任铁道兵后勤部长、国防部办公厅副主任、解放军总参管理局局长。                                                                                                                                                                                                                                                                                                                           

In [8]:
import pandas as pd
df=pd.read_csv("解放将军信息_02.csv",encoding="utf-8")
df

,Unnamed: 0,中文名,出生日期,逝世日期,出生地,毕业院校,北伐战争,土地革命,抗日战争,解放战争,解放后
0,0,朱德,1886年12月1日,1976年7月6日,四川省仪陇县,无,无,1月，和毛泽东率红四军主力离开井冈山，向赣南、闽西进军。以武装斗争发动土地革命，建立红色政权...,在中共七大上讲演1937年7月抗日战争爆发后，任国民革命军第八路军总指挥（不久改称国民革命军...,在解放战争中，任中国人民解放军总司令。1945年4月，朱德同志在党的七大上作《论解放区战场》...,无
1,1,彭德怀,1898年10月24日,1974年11月29日,湖南省湘潭县石潭镇乌石寨,湖南陆军讲武堂,1926年，彭德怀随部队编入国民革命军，参加北伐战争，结识共产党人段德昌，开始接受共产主义思...,无,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,无
2,2,林彪,1907年12月5日,1971年9月13日,None,None,在黄埔军校由共青团转入中国共产党，并任3连中共支部书记。1926年10月毕业后，由广州北上武...,战役时间：土地革命战争时期的1930年11月5日至1931年1月3日，历时60天。战役时间：...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...
3,3,刘伯承,1892年12月4日,1986年10月7日,重庆市开州区,None,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,刘伯承与毛泽东主席天安门城楼上1949年4月参与指挥渡江战役，实施京沪杭会战，并亲自指挥第二...
4,4,贺龙,1896年3月22日,1969年6月9日,湖南桑植,无,1924年至1927年中国第一次国内革命战争期间，贺龙积极拥护孙中山先生“联俄、联共、扶助农...,土地革命战争时期，1927年参加领导南昌起义，任起义军总指挥，同年加入中国共产党。,长征胜利结束后的贺龙抗日战争开始后，红军改编为国民革命军第八路军，贺龙任八路军第120师师长...,贺龙被任命为一二零师师长日军投降后，贺龙率领晋绥部队主力挥师北上，解放了晋中广大地区，并与聂...,1949年12月，贺龙率华北野战军第十八兵团等部，由陕入川，配合刘伯承、邓小平指挥的第二野战...
...,...,...,...,...,...,...,...,...,...,...,...
943,1036,赖光勋,1914年11月,2003年1月25日17时,None,无,无,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,红军地图奇缺，有的地区甚至没有一份地图，赖光勋带领测绘科在极端艰苦的条件下，克服重重困难，千...
944,1037,黎光,1914年,无,四川省南充市仪陇县,None,无,无,抗日战争开始后，黎光任八路军129师386旅771团1营1连排长。在和日军作战中，一颗子弹从...,无,无
945,1038,汪易,1916年,1995年2月13日,四川省阆中市,无,无,无,无,汪易同志在解放战争时期，曾在承德、赤峰、热东、热辽的广大地区战斗、工作了3年时间，他在热河和...,无
946,1040,孙光,1961年6月21日,无,西安,无,无,无,无,无,无


In [9]:
for i in range(len(df)):
    df["出生日期"].values[i]=df["出生日期"].values[i][:5]
    df["逝世日期"].values[i]=df["逝世日期"].values[i][:5]
df.to_csv("解放将军信息_03")

,Unnamed: 0,中文名,出生日期,逝世日期,出生地,毕业院校,北伐战争,土地革命,抗日战争,解放战争,解放后
0,0,朱德,1886年,1976年,四川省仪陇县,无,无,1月，和毛泽东率红四军主力离开井冈山，向赣南、闽西进军。以武装斗争发动土地革命，建立红色政权...,在中共七大上讲演1937年7月抗日战争爆发后，任国民革命军第八路军总指挥（不久改称国民革命军...,在解放战争中，任中国人民解放军总司令。1945年4月，朱德同志在党的七大上作《论解放区战场》...,无
1,1,彭德怀,1898年,1974年,湖南省湘潭县石潭镇乌石寨,湖南陆军讲武堂,1926年，彭德怀随部队编入国民革命军，参加北伐战争，结识共产党人段德昌，开始接受共产主义思...,无,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,彭德怀于1928年加入中国共产党，第二次国内革命战争时期担任中国工农红军师长、军长、三军团总...,无
2,2,林彪,1907年,1971年,None,None,在黄埔军校由共青团转入中国共产党，并任3连中共支部书记。1926年10月毕业后，由广州北上武...,战役时间：土地革命战争时期的1930年11月5日至1931年1月3日，历时60天。战役时间：...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...,1925年参加中国共产党；在井冈山时期先后任营长、团长、军长、军团长等职；抗日战争时期任八路...
3,3,刘伯承,1892年,1986年,重庆市开州区,None,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,辛亥革命时期从军，1926年加入中国共产党。相继参加了北伐战争、八一南昌起义、土地革命战争、...,刘伯承与毛泽东主席天安门城楼上1949年4月参与指挥渡江战役，实施京沪杭会战，并亲自指挥第二...
4,4,贺龙,1896年,1969年,湖南桑植,无,1924年至1927年中国第一次国内革命战争期间，贺龙积极拥护孙中山先生“联俄、联共、扶助农...,土地革命战争时期，1927年参加领导南昌起义，任起义军总指挥，同年加入中国共产党。,长征胜利结束后的贺龙抗日战争开始后，红军改编为国民革命军第八路军，贺龙任八路军第120师师长...,贺龙被任命为一二零师师长日军投降后，贺龙率领晋绥部队主力挥师北上，解放了晋中广大地区，并与聂...,1949年12月，贺龙率华北野战军第十八兵团等部，由陕入川，配合刘伯承、邓小平指挥的第二野战...
...,...,...,...,...,...,...,...,...,...,...,...
943,1036,赖光勋,1914年,2003年,None,无,无,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,赖光勋（1914年-2003年1月25日），福建永定人，1930年参加红军，1932年加入共...,红军地图奇缺，有的地区甚至没有一份地图，赖光勋带领测绘科在极端艰苦的条件下，克服重重困难，千...
944,1037,黎光,1914年,无,四川省南充市仪陇县,None,无,无,抗日战争开始后，黎光任八路军129师386旅771团1营1连排长。在和日军作战中，一颗子弹从...,无,无
945,1038,汪易,1916年,1995年,四川省阆中市,无,无,无,无,汪易同志在解放战争时期，曾在承德、赤峰、热东、热辽的广大地区战斗、工作了3年时间，他在热河和...,无
946,1040,孙光,1961年,无,西安,无,无,无,无,无,无
